In [27]:
import pandas as pd

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier


In [9]:
df_health = pd.read_csv ('healthcare_dataset.csv')

In [10]:
df_health. head ()

,Patient_ID,Age,Gender,Insurance_Type,Chronic_Disease,Mental_Health_Status,Employment_Status,Education_Level,Transportation_Access,Distance_from_Facility,Area_Type,Booking_Date,Appointment_Date,Appointment_Outcome
0,75763,93.0,Male,Medicare,False,True,Student,Advanced Degree,True,37.785038,Suburban,2023-11-04,2024-08-15,Show
1,84945,52.0,Male,Uninsured,False,False,Employed,College,True,NaN,Rural,2023-10-28,2024-07-11,No-Show
2,88559,8.0,Male,Medicare,True,NaN,NaN,Advanced Degree,True,27.277654,Rural,2023-02-12,2024-10-26,Show
3,72217,5.0,Other,Medicare,False,True,Student,College,True,44.618708,NaN,2023-01-30,2024-01-31,Show
4,44071,76.0,Male,Uninsured,True,False,Employed,Advanced Degree,False,12.568497,Urban,2023-11-27,2024-03-03,NaN


In [13]:
df_health ['Booking_Date']=pd.to_datetime (df_health['Booking_Date'])
df_health ['Appointment_Date']=pd.to_datetime (df_health['Appointment_Date'])
df_health['date_difference']=(df_health ['Appointment_Date']-df_health ['Booking_Date'])
df_health['date_difference'] = df_health['date_difference'].dt.days
#df_health.head()
df_health. info ()

<class 'pandas.core.frame.DataFrame'>
Index: 11739 entries, 0 to 49998
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Patient_ID              11739 non-null  int64         
 1   Age                     11739 non-null  float64       
 2   Gender                  11739 non-null  object        
 3   Insurance_Type          11739 non-null  object        
 4   Chronic_Disease         11739 non-null  object        
 5   Mental_Health_Status    11739 non-null  object        
 6   Employment_Status       11739 non-null  object        
 7   Education_Level         11739 non-null  object        
 8   Transportation_Access   11739 non-null  object        
 9   Distance_from_Facility  11739 non-null  float64       
 10  Area_Type               11739 non-null  object        
 11  Booking_Date            11739 non-null  datetime64[ns]
 12  Appointment_Date        11739 non-null  datetime64[

In [31]:
#X and y
df_health['day_of_week'] = df_health['Appointment_Date'].dt.day_name()

# X and y
X = df_health.drop(['Patient_ID', 'Booking_Date', 'Appointment_Date', 'Appointment_Outcome'], axis=1)
y = df_health['Appointment_Outcome']

# Split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess numerical and categorical features separately
numeric_features = ['Age', 'Distance_from_Facility', 'date_difference']
categorical_features = ['Gender', 'Insurance_Type',  'Mental_Health_Status', 
                         'Employment_Status', 'Education_Level', 'Transportation_Access', 
                         'Area_Type', 'day_of_week']

# Create transformers for preprocessing
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Initialize  within a pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

Accuracy: 0.4885008517887564
Confusion Matrix:
 [[596 609]
 [592 551]]
Classification Report:
               precision    recall  f1-score   support

     No-Show       0.50      0.49      0.50      1205
        Show       0.47      0.48      0.48      1143

    accuracy                           0.49      2348
   macro avg       0.49      0.49      0.49      2348
weighted avg       0.49      0.49      0.49      2348



In [35]:
# Separate features and target variable
X = df_health.drop('Appointment_Outcome', axis=1)
y = df_health['Appointment_Outcome']

# X and y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# numerical and categorical 
numeric_features = ['Age', 'Distance_from_Facility', 'date_difference']
categorical_features = ['Gender', 'Insurance_Type', 'Mental_Health_Status', 
                         'Employment_Status', 'Education_Level', 'Transportation_Access', 
                         'Area_Type', 'day_of_week']

# Preprocess numerical features
numeric_transformer = StandardScaler()

# Preprocess categorical features
categorical_transformer = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(drop='first'), categorical_features)]
)

# Combine 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Initialize  within a pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

model.fit(X_train, y_train)

#  predictions
y_pred = model.predict(X_test)

# accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5017035775127768


In [36]:
# Evaluation
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

Accuracy: 0.5017035775127768
Confusion Matrix:
 [[615 560]
 [610 563]]
Classification Report:
               precision    recall  f1-score   support

     No-Show       0.50      0.52      0.51      1175
        Show       0.50      0.48      0.49      1173

    accuracy                           0.50      2348
   macro avg       0.50      0.50      0.50      2348
weighted avg       0.50      0.50      0.50      2348

